In [1]:
import os 
import csv
import os
import csv
import sys
sys.path.append('../..')

import json
import random
import workFactory as wf

from tqdm import tqdm
from argparse import ArgumentParser

csv.field_size_limit(sys.maxsize)

131072

In [2]:
def saveFM_Lexicon(works, matchWords, savePath):
    writeTarget = []
    for work in tqdm(works):
        label = work['label']
        num, paragraphs = firstMatch_Lexicon(work, matchWords)
        writeTarget.append({
            'label': label,
            'match': num,
            'contents': paragraphs
        })
    with open(f'{savePath}', 'w+', encoding='utf-8') as f:
        json.dump(writeTarget, f)

In [3]:
target = 42

workPath = f'../tsv/morpheme/{target}'
savePath = f'./tsv/fm-c-d/{target}'

os.makedirs(savePath, exist_ok=True)
dev = wf.load_tsv_dataset(f'{workPath}/dev.tsv')

work = dev[10]

labels = wf.load_tsv_label('../numeric_label.tsv')
matchWords = labels[target-1]['name']

In [59]:
split = work['content'].replace('\n', '').split()
chunks = list(seperate(split, len(split)//10))

In [60]:
print(len(chunks))
for c in chunks:
    print(len(c))

10
2990
2990
2990
2990
2990
2990
2990
2990
2990
2990


In [4]:
def seperate(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [149]:
def firstMatch_chunk(chunk, matchWords):
    for index, word in enumerate(chunk):
        if word in matchWords:
            l_paragraph = chunk[:index]
            r_paragraph = chunk[index:]
            ct = 0
            paragraph = []
            for wd in r_paragraph:
                ct += len(wd)
                paragraph.append(wd)
                if ct > 512:
                    #print('success', paragraph[:5])
                    return 'success at r_list', 'r', ' '.join(paragraph[:-1])
            if ct < 512:
                for wd in reversed(l_paragraph):
                    ct += len(wd)
                    paragraph.insert(0, wd)
                    if ct > 512:
                        #print('success at rList', paragraph[:5])
                        return 'success at l_list', 'l', ' '.join(paragraph[1:])
    ct = 0
    for idx, wd in enumerate(chunk):
        ct += len(wd)
        if ct > 512:
            #print('nomatch', chunk[:10])
            return 'no match', 'n', ' '.join(chunk[:idx])

In [150]:
def firstMatch_work(work, matchWords, sepNum):
    paragraphs = []
    split = work['content'].replace('\n', '').split()
    chunks = list(seperate(split, len(split)//sepNum))
    for c in chunks[:sepNum]:
        s, c, p = firstMatch_chunk(c, matchWords)
        paragraphs.append({'status': s, 'code': c, 'paragraph': p})
        
    return paragraphs

In [151]:
def saveFMtoJSON_chunk(works, matchWords, savePath, sepNum):
    writeTarget = []
    for work in tqdm(works):
        label = work['label']
        if len(work['content'].replace(' ', ''))//sepNum > 512:
            paragraphs = firstMatch_work(work, matchWords, sepNum)
            writeTarget.append({
                'label': label,
                'contents': paragraphs
            })
        else:
            print(f"Size Error:\n\twork size {len(work['content'].replace(' ', ''))}, should have at least {sepNum*512}, skip work")
    with open(f'{savePath}', 'w+', encoding='utf-8') as f:
        json.dump(writeTarget, f)

In [152]:
target = 42
sepNum = 10

workPath = f'../tsv/morpheme/{target}'
savePath = f'./tsv/fm-c-scatter/{target}/result.json'
os.makedirs(f'./tsv/fm-c-scatter/{target}', exist_ok=True)

works = wf.load_tsv_dataset(f'{workPath}/dev.tsv')

labels = wf.load_tsv_label('../numeric_label.tsv')
matchWords = labels[target-1]['name']

saveFMtoJSON_chunk(works[:15], matchWords, savePath, sepNum)

100%|███████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 61.22it/s]

Size Error:
	work size 1584, should have at least 5120, skip work
Size Error:
	work size 1037, should have at least 5120, skip work


In [86]:
for work in test[:10]:
    split = work['content'].replace('\n', '').split()
    chunks = list(seperate(split, len(split)//10))
    for i, c in enumerate(chunks[:10]):
        if i+1 < len(chunks):
            chunk = c+chunks[i+1]
        else:
            chunk = c    
        print(firstMatch_chunk(chunk, matchWords))
    print('*'*20)

NameError: name 'test' is not defined

In [29]:
lst = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
r_lst = lst[10:]
l_lst = lst[:10]
print(l_lst)
print(r_lst)

for id, it in enumerate(l_lst):
    print(id, l_lst[-id-1])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10]
0 9
1 8
2 7
3 6
4 5
5 4
6 3
7 2
8 1
9 0


In [166]:
for work in works[:100]:
    wlen = len(work['content'].replace('\n', '').replace(' ', ''))
    if wlen < 5120:
        print('!', wlen, work['content'].replace('\n', '').replace(' ', ''))

! 1524 この度は征夷大将軍就任のこと、誠に慶賀の至りに存じます唐風の衣裳に身を包んだ琉球王子は平伏したまま低い声で続けた我が琉球国より心よりお祝い申し上げます海を越えて江戸城に現れた異国風の使節の中でも一際高雅な装いだが、体格はまだ背も伸び切らず、語る声にも熟しきらぬ幼さが漂う月からの使い?上座からいとけない声が飛んでくる琉球ですわまだ若く美しい乳母が穏やかに訂正する海を越えた琉球から上様が将軍になられたお祝いに向こうの若君が来て下さったのです広間の最上段の、身に着けた豪奢な絹の着物に半ば埋もれるようにして座している四歳の将軍は小さな白い手で下座の王子を指差すでも、おが見せてくれた絵巻物の月の使いにそっくりだよ幼子の白桃じみたふっくりした両の頬に笑窪が刻まれる月の王子様だからあんなに美しいんだろ居並ぶ幕臣と琉球使節団の人々の表情がふと和らいだ平伏した琉球王子も浅黒い端正な面に微苦笑を浮かべるこちらも十四、五歳で上座の将軍と比べてもまだ兄ほどの少年である確かにご立派な若君でいらっしゃいますねお幸と呼ばれた乳母も笑顔で頷く月の国はいかなる所か幼い将軍は着膨れした錦の着物から抜き出たか細い首を傾げる我が琉球は瑠璃色の澄んだ海にの浜に囲まれた小さな島国にございますまだ少年の琉球王子は控え目だが誇りやかな声で続けたこちらの春には見事な桜が花開きますが、琉球ではでいごが春に朱い花を咲かせます唐風の装いを纏う若き王子と使節の人々の目に懐かしげな光が宿るでいご?幼子は不思議そうに問い返す南国の花にございます琉球ではの桜が見られない代わりに真っ赤なでいごが春を告げるのでございます見てみたいのう小さな体に絹の和服を重ねて着けた征夷大将軍は傍らの乳母にあどけない声で語った余も琉球に行ってみたい居並ぶ幕臣たちは困惑した笑いを浮かべて互いを見合わせるの城はこちらのお城ほど大きく立派ではございませぬ琉球の王子はこちらもまだ少年の幼さの残る顔に寂しい微笑を漂わせたしかし、上様がお越しになるようなことがあれば我々は心より歓迎致しますきっと行く、余は必ず月の国に参るぞ広間に幼子の高らかに笑う声が響き渡った＊＊＊＊＊時下、征夷大将軍様に置かれましてはますますご健勝のこととお慶び申し上げます、広い肩に大陸風の衣装を纏い、蜜色の肌も精悍な琉球王子は朗々たる声で述べるこの度は我が父の琉球国王即位に伴い